# This code prepares the multi-class image data

In [ ]:
# First lets connect the Gdrive that contains the data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
# The path below should point to the directory containing this notebook and the associated utility files
# Change it if necessary
os.chdir('/content/drive/MyDrive/U-net Multi-class/diaretdb1_v_1_1/resources/')
!ls

example_evalresults  images  testdatasets  train
html		     test    toolkit	   traindatasets


# Lets automatically partition the data into train and test folders. Image number 1-27 in train and remaining in test

In [ ]:

if not os.path.exists('./train/'):
    os.makedirs('./train/')
if not os.path.exists('./train/images'):
    os.makedirs('./train/images')
if not os.path.exists('./train/GT/'):
    os.makedirs('./train/GT/')



if not os.path.exists('./test/'):
    os.makedirs('./test/')
if not os.path.exists('./test/images'):
    os.makedirs('./test/images')


In [ ]:
import numpy as np
import os
import skimage.io as io
import skimage.transform as trans

In [ ]:
def prepare_multi_class_GT(GT_PATH, class_names, savepath, target_size=(512,512), n_class=3):

    f_names = os.listdir(GT_PATH+class_names[0])
    for files in f_names:
        GT_im=np.zeros(np.concatenate((target_size,n_class),axis=None)) #This creates a zero array of size (512,512,3)
        FG=np.zeros(target_size)
        
        for idx,cn in enumerate(class_names):
            lab=io.imread(GT_PATH+cn+files, as_gray=True)
            lab = trans.resize(lab,target_size)
            if(np.max(lab)>1):
                lab=lab/255
            lab[lab>=0.1]=1 #threshold at 0.1. Change this value based on your requirement
            lab[lab<0.1]=0
            
            if (idx<2): #Bright Lesions
                GT_im[:,:,1]=GT_im[:,:,1]+lab
            else:#Red Lesions
                GT_im[:,:,0]=GT_im[:,:,0]+lab
        if(np.sum(GT_im[:,:,0])>0):
            GT_im[:,:,0]=GT_im[:,:,0]/np.ptp(GT_im[:,:,0])
        if(np.sum(GT_im[:,:,1])>0):
            GT_im[:,:,1]=GT_im[:,:,1]/np.ptp(GT_im[:,:,1])
        FG=(GT_im[:,:,0]+GT_im[:,:,1]>0).astype(int)
        GT_im[:,:,2]=1-FG
        io.imsave(savepath+files,GT_im)
       

In [ ]:

GT_PATH='./images/ddb1_groundtruth/'
class_names=['hemorrhages/','redsmalldots/','hardexudates/', 'softexudates/',]


if not os.path.exists('./images/GT/'):
    os.makedirs('./images/GT/')

prepare_multi_class_GT(GT_PATH,class_names, './images/GT/')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: ./GT/image051.png is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: ./GT/image057.png is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: ./GT/image072.png is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: ./GT/image069.png is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: ./GT/image050.png is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: ./GT/image056.png is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: ./GT/image028.png is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: ./GT/image049.png is a low contrast image
/usr/local/lib/python3.7/dist-packages/i

# Now your data is ready. Partition it to 'train' and 'test' folders using the code below

In [ ]:
import shutil
for i in range(1,90):
  if (i<10):
     shutil.copy('./images/ddb1_fundusimages/image00'+str(i)+'.png', './train/images/')
     shutil.copy('./images/GT/image00'+str(i)+'.png', './train/GT/')
  elif (i>=10 and i<28):
      shutil.copy('./images/ddb1_fundusimages/image0'+str(i)+'.png', './train/images/')
      shutil.copy('./images/GT/image0'+str(i)+'.png', './train/GT/')
  else:
    shutil.copy('./images/ddb1_fundusimages/image0'+str(i)+'.png', './test/images/')

